In [1]:
import re
import pickle
import json
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import math
from difflib import SequenceMatcher
import pandas as pd
from statistics import stdev
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
section_labels = pickle.load(open("../pickles_data/section_labels.pkl", "rb"))

In [3]:
def get_citations(folder) :
    citation_list = {}
    for file in os.listdir(folder) :
        tree = ET.parse(folder+file)
        root = tree.getroot()
        id = file[:8]
        for element in root.iterfind("algorithm"):
            if(element.attrib['name']=="ParsCit"):
                citlist = element.getchildren()
                cits = citlist[0].getchildren()
                citations = []
                for cit in cits:
                    cit_dict = {}
                    if(cit.attrib['valid']=="true"):
                        try :
                            title = cit.find('title').text.lower()
                        except :
                            title = cit.find('rawString').text.lower()
                        cit_dict['title'] = title
                        cit_dict['cit'] = cit
                        cit_dict['context'] = " ".join([context.text.lower() for context in cit.findall('contexts/context')])
                        citations.append(cit_dict)
                
                citation_list[id] = citations
            
    return citation_list 
    

In [4]:
citations = get_citations("../xmls/")

In [19]:
ind_to_pap = {}
pap_to_ind = {}
contexts = []
title = []
paper_title = []
paper_abs = []
paper_conc = []
paper_rest = []
paper_intro = []

count = 0
for key in citations :
    papers = citations[key]
    section = section_labels[key]
    for paper in papers :
        contexts.append(paper['context'])
        title.append(paper['title'])
        abstract = section['abstract']['overall'].lower()
        conc = section['conclusion']['overall'].lower()
        intro = section['introduction']['overall'].lower()
        paper_abs.append(abstract)
        paper_conc.append(conc)
        paper_intro.append(intro)
        full_text = section['overall']
        full_text = full_text.replace(abstract, "")
        full_text = full_text.replace(conc, "")
        full_text = full_text.replace(intro, "")
        paper_rest.append(full_text)
        ind_to_pap[count] = key+"_"+paper['title']
        pap_to_ind[key+"_"+paper['title']]=count
        count+=1

In [6]:
print(count)
print(len(contexts))
print(len(title))
print(len(paper_abs))
print(len(paper_conc))
print(len(paper_rest))

31943
31943
31943
31943
31943
31943


In [7]:
print(list(section_labels.keys())[0])

P10-1026


In [8]:
section_labels['P10-1026'].keys()

dict_keys(['overall', 'experiment', 'conclusion', 'discussion', 'other_sections', 'related_work', 'abstract', 'introduction'])

In [61]:
raw_matrix = TfidfVectorizer(stop_words = 'english', min_df = 0.01, token_pattern='[a-zA-Z]+')

In [62]:
corpus = []
corpus.extend(contexts)
corpus.extend(paper_rest)
print(len(corpus))

63886


In [63]:
matrix = raw_matrix.fit_transform(corpus)

In [64]:
outputs = []
for i in range(31943) :
    outputs.append(cosine_similarity(matrix[i],matrix[i+31943]))

print(len(outputs))

31943


In [65]:
pickle.dump(outputs, open("pickles/alf_sim_con_rest_out.pkl", "wb"))

In [66]:
dataset = {}
count = 0
for i in range(len(outputs)) :
    pap = ind_to_pap[i]
    key = pap.split("_")[0]
    name = pap.split("_")[1]
    dict1 = {}
    dict1['paper_name'] = name
    dict1['data'] = outputs[i]
    if(key not in dataset) :
        dataset[key] = []
    dataset[key].append(dict1)
    count+=1
    if(count%1000==0) :
        print(count) 

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


In [26]:
print(len(dataset.keys()))

1172


In [17]:
print(len(citations.keys()))

1181


In [67]:
pickle.dump(dataset, open("pickles/alf_sim_con_rest_data.pkl", "wb"))